In [ ]:
import json
from transformers import RobertaTokenizer, RobertaModel
# from transformers import AutoTokenizer, RobertaModel
import numpy as np
import argparse
import math
import torch
import torch.nn.functional as F
from tqdm import trange
import datetime
import os, random
from torch.utils.tensorboard import SummaryWriter

# 创建一个TensorBoard写入器
writer = SummaryWriter()

torch.cuda.set_device(0)

# Tokenizer

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--max_sequence_len', type=int, default=100, help='max length of the tagging matrix')
parser.add_argument('--sentiment2id', type=dict, default={'negative': 2, 'neutral': 3, 'positive': 4}, help='mapping sentiments to ids')
parser.add_argument('--model_name_or_path', type=str, default='roberta-base', help='reberta model')
parser.add_argument('--batch_size', type=int, default=16, help='json data path')
parser.add_argument('--device', type=str, default="cuda", help='gpu or cpu')
parser.add_argument('--prefix', type=str, default="./data/", help='dataset and embedding path prefix')

parser.add_argument('--data_version', type=str, default="D1", choices=["D1", "D2"], help='dataset and embedding path prefix')
parser.add_argument('--dataset', type=str, default="res14", choices=["res14", "lap14", "res15", "res16"], help='dataset')

parser.add_argument('--bert_feature_dim', type=int, default=768, help='dimension of pretrained bert feature')
parser.add_argument('--epochs', type=int, default=2000, help='training epoch number')
parser.add_argument('--class_num', type=int, default=5, help='label number')
parser.add_argument('--task', type=str, default="triplet", choices=["pair", "triplet"], help='option: pair, triplet')
parser.add_argument('--model_dir', type=str, default="savemodel/", help='model path prefix')
parser.add_argument('--log_path', type=str, default="log.log", help='log path')


args = parser.parse_known_args()[0]

#加载预训练字典和分词方法
tokenizer = RobertaTokenizer.from_pretrained(
    pretrained_model_name_or_path=args.model_name_or_path,  # 'bert-base-chinese'
    cache_dir='./',  # 将数据保存到的本地位置，使用cache_dir 可以指定文件下载位置
    force_download=False
)


args.log_path = 'log_{}_{}.log'.format(args.data_version, args.dataset)
def logging(info, file_name=args.log_path):
    with open(file_name, 'a+') as f:
        print(info, file=f)
    print(info)

def seed_torch(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed(seed)
    random.seed(seed)
    
seed = 666
seed_torch(seed)


logging("\n\n=========\nseed: {}\n=========\n".format(seed))

# # # word = '华为云FusionInsight三次蝉联中国大数据市场第一'
# word = 'predict The sushi . i think .'
# tokenizer.encode(word, add_special_tokens=False)



seed: 666



# Data

In [ ]:
class Instance(object):
    '''
    Re-organiztion for a single sentence;
    Input is in the formulation of: 
        {
        'id': '3547',
        'sentence': 'Taj Mahal offeres gret value and great food .',
        'triples': [
                    {'uid': '3547-0',
                    'target_tags': 'Taj\\O Mahal\\O offeres\\O gret\\O value\\B and\\O great\\O food\\O .\\O',
                    'opinion_tags': 'Taj\\O Mahal\\O offeres\\O gret\\O value\\O and\\O great\\B food\\O .\\O',
                    'sentiment': 'positive'},
                    {'uid': '3547-1',
                    'target_tags': 'Taj\\O Mahal\\O offeres\\O gret\\O value\\O and\\O great\\O food\\B .\\O',
                    'opinion_tags': 'Taj\\O Mahal\\O offeres\\O gret\\O value\\O and\\O great\\B food\\O .\\O',
                    'sentiment': 'positive'}
                    ]
        }
    Usage example:
    # sentence_packs = json.load(open(args.prefix + args.dataset + '/test.json'))
    # instances = load_data_instances(sentence_packs, args)
    # testset = DataIterator(instances, args)
    '''
    def __init__(self, tokenizer, single_sentence_pack, args):
        self.id = single_sentence_pack['id']
        self.sentence = single_sentence_pack['sentence']
        self.tokens = tokenizer.tokenize(self.sentence, add_prefix_space=True)  # ['ĠL', 'arg', 'est', 'Ġand', 'Ġfres', 'hest', 'Ġpieces', 'Ġof', 'Ġsushi', 'Ġ,', 'Ġand', 'Ġdelicious', 'Ġ!']
        self.word_spans = self.get_word_spans(tokens=self.tokens)  # '[[0, 2], [3, 3], [4, 5], [6, 6], [7, 7], [8, 8], [9, 9], [10, 10], [11, 11], [12, 12]]'
        assert len(self.sentence.strip().split(' ')) == len(self.word_spans)
        self.triplets = single_sentence_pack['triples']

        self.bert_tokens = tokenizer.encode(self.sentence, add_special_tokens=False, add_prefix_space=True)
        self.bert_tokens_padded = torch.zeros(args.max_sequence_len).long()
        
        self.mask = torch.ones((args.max_sequence_len, args.max_sequence_len))
        self.mask[:, len(self.bert_tokens):] = 0
        self.mask[len(self.bert_tokens):, :] = 0
        for i in range(len(self.bert_tokens)):
            self.mask[i][i] = 0

        if len(self.bert_tokens) != self.word_spans[-1][-1] + 1:
            print(self.sentence, self.word_spans)
            
        for i in range(len(self.bert_tokens)):
            self.bert_tokens_padded[i] = self.bert_tokens[i]
        
        self.tagging_matrix = self.get_tagging_matrix(triplets=self.triplets, word_spans=self.word_spans, sentiment2id=args.sentiment2id)
        self.tagging_matrix = (self.tagging_matrix + self.mask - torch.tensor(1)).long()
        
    def get_word_spans(self, tokens):
        '''
        get roberta-token-spans of each word in a single sentence
        according to the rule: each 'Ġ' maps to a single word
        required: tokens = tokenizer.tokenize(sentence, add_prefix_space=True)
        '''

        l_indx = 0
        r_indx = 0
        word_spans = []
        c = 0
        while r_indx + 1 < len(tokens):
            if tokens[r_indx+1][0] == 'Ġ':
                word_spans.append([l_indx, r_indx])
                r_indx += 1
                l_indx = r_indx
            else:
                r_indx += 1
        word_spans.append([l_indx, r_indx])
        return word_spans
    
    def get_spans_from_BIO(self, tags, token_ranges):
        '''for BIO tag'''
        tags = tags.strip().split()
        length = len(tags)
        spans = []
        # start = -1
        for i in range(length):
            # print(i)
            if tags[i].endswith('B'):
                spans.append(token_ranges[i])
                # 遇到一个 B，意味着开启一个新的三元组
                # 接下来需要看有没有跟 I，如果一直跟着 I，则一直扩大这个三元组的span范围，直到后面是 O 或下一个 B为止，此时则重新继续找下一个 B
                # 其实如果一旦找到一个 B 然后直到这个 B 终止于一个 O或下一个 B，这个刚刚被找到的三元组 BI··O/B的范围内就不再会有其他 B，因此不需要回溯，直接 go on
            elif tags[i].endswith('I'):
                spans[-1][-1] = token_ranges[i][-1]
            else:  # endswith('O')
                pass
        return spans
        
    def get_tagging_matrix(self, triplets, word_spans, sentiment2id = {'negative': 2, 'neutral': 3, 'positive': 4}):
        '''
        mapping the tags to a Matrix Tagginh scheme
        '''
        tagging_matrix = torch.zeros((args.max_sequence_len, args.max_sequence_len))
        '''
        tagging_matrix                      O   
        array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        A      [0., 0., 0., 0., 0., 0., 0., S , 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
        '''

        for triplet in triplets:
            # triplets = [{'uid': '2125-0',
            #   'target_tags': 'Largest\\O and\\O freshest\\O pieces\\B of\\I sushi\\I ,\\O and\\O delicious\\O !\\O',
            #   'opinion_tags': 'Largest\\B and\\O freshest\\B pieces\\O of\\O sushi\\O ,\\O and\\O delicious\\B !\\O',
            #   'sentiment': 'positive'}]

            # print(aspect_tags)
            # print(opinion_tags)
            # print(sentiment_tags)
            # Largest\O and\O freshest\O pieces\B of\I sushi\I ,\O and\O delicious\O !\O
            # Largest\B and\O freshest\B pieces\O of\O sushi\O ,\O and\O delicious\B !\O
            # positive
            # break
            # sentiment2id = {'negative': 2, 'neutral': 3, 'positive': 4}

            # aspect = triple['target_tags']
            # opinion = triple['opinion_tags']
            aspect_tags = triplet['target_tags']
            opinion_tags = triplet['opinion_tags']
            sentiment = triplet['sentiment']

            aspect_spans = self.get_spans_from_BIO(aspect_tags, self.word_spans)
            opinion_spans = self.get_spans_from_BIO(opinion_tags, self.word_spans)

            '''set tag for sentiment'''
            for aspect_span in aspect_spans:
                for opinion_span in opinion_spans:
                    # print(aspect_span)
                    # print(opinion_span)
                    al = aspect_span[0]
                    ar = aspect_span[1]
                    pl = opinion_span[0]
                    pr = opinion_span[1]
                    for i in range(al, ar+1):
                        for j in range(pl, pr+1):
                            # print(al, ar, pl, pr)
                            # print(i, j)
                            # print(i==al and j==pl)
                            if i==al and j==pl:
                                tagging_matrix[i][j] = sentiment2id[sentiment]  # 3 4 5
                            else:
                                tagging_matrix[i][j] = 1  # 1: ctd

        return tagging_matrix

    
def load_data_instances(tokenizer, sentence_packs, args):
    instances = []
    for sentence_pack in sentence_packs:
        instances.append(Instance(tokenizer, sentence_pack, args))
    return instances


class DataIterator(object):
    def __init__(self, instances, args):
        self.instances = instances
        self.args = args
        self.batch_count = math.ceil(len(instances)/args.batch_size)

    def get_batch(self, index):
        sentence_ids = []
        word_spans = []
        bert_tokens = []
        masks = []
        tagging_matrices = []
        tokenized = []

        for i in range(index * self.args.batch_size, min((index + 1) * self.args.batch_size, len(self.instances))):
            sentence_ids.append(self.instances[i].id)
            word_spans.append(self.instances[i].word_spans)
            bert_tokens.append(self.instances[i].bert_tokens_padded)
            masks.append(self.instances[i].mask)
            # aspect_tags.append(self.instances[i].aspect_tags)
            # opinion_tags.append(self.instances[i].opinion_tags)
            tagging_matrices.append(self.instances[i].tagging_matrix)
            tokenized.append(self.instances[i].tokens)

        bert_tokens = torch.stack(bert_tokens).to(self.args.device)
        # lengths = torch.tensor(lengths).to(self.args.device)
        masks = torch.stack(masks).to(self.args.device)
        # aspect_tags = torch.stack(aspect_tags).to(self.args.device)
        # opinion_tags = torch.stack(opinion_tags).to(self.args.device)
        tagging_matrices = torch.stack(tagging_matrices).long().to(self.args.device)
        return sentence_ids, bert_tokens, masks, word_spans, tagging_matrices, tokenized

# Load Dataset
train_sentence_packs = json.load(open(os.path.abspath(args.prefix + args.data_version + '/' + args.dataset + '/train.json')))
# random.shuffle(train_sentence_packs)
dev_sentence_packs = json.load(open(os.path.abspath(args.prefix + args.data_version + '/' + args.dataset + '/dev.json')))
test_sentence_packs = json.load(open(os.path.abspath(args.prefix + args.data_version + '/' + args.dataset + '/test.json')))

train_instances = load_data_instances(tokenizer, train_sentence_packs, args)
dev_instances = load_data_instances(tokenizer, dev_sentence_packs, args)
test_instances = load_data_instances(tokenizer, test_sentence_packs, args)

trainset = DataIterator(train_instances, args)
devset = DataIterator(dev_instances, args)
testset = DataIterator(test_instances, args)

# Model

In [ ]:
# import torch.nn as nn

class TokenComparisonModel(torch.nn.Module):
    def __init__(self, num_classes, embedding_dim):
        super(TokenComparisonModel, self).__init__()
        self.num_classes = num_classes
        self.embedding_dim = embedding_dim
        self.embedding_layer = nn.Embedding(num_classes, embedding_dim)
        self.fc_layer = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, input_tokens, target_tokens, negative_tokens):
        input_embeddings = self.embedding_layer(input_tokens)
        target_embeddings = self.embedding_layer(target_tokens)
        negative_embeddings = self.embedding_layer(negative_tokens)
        input_embeddings = self.fc_layer(input_embeddings)
        target_embeddings = self.fc_layer(target_embeddings)
        negative_embeddings = self.fc_layer(negative_embeddings)
        input_target_similarity = nn.functional.cosine_similarity(input_embeddings, target_embeddings)
        input_negative_similarity = nn.functional.cosine_similarity(input_embeddings, negative_embeddings)
        return input_target_similarity, input_negative_similarity

In [ ]:
class MultiInferBert(torch.nn.Module):
    def __init__(self, args):
        super(MultiInferBert, self).__init__()

        self.args = args
        self.bert = RobertaModel.from_pretrained("roberta-base")
        self.norm0 = torch.nn.LayerNorm(args.bert_feature_dim)
        self.drop_feature = torch.nn.Dropout(0.1)

        self.linear1 = torch.nn.Linear(args.bert_feature_dim*2, self.args.max_sequence_len)
        self.norm1 = torch.nn.LayerNorm(self.args.max_sequence_len)
        self.cls_linear = torch.nn.Linear(self.args.max_sequence_len, args.class_num)
        self.cls_linear1 = torch.nn.Linear(self.args.max_sequence_len, 2)
        self.gelu = torch.nn.GELU()

    def forward(self, tokens, masks):
        bert_feature, _ = self.bert(tokens, masks, return_dict=False)
        bert_feature = self.norm0(bert_feature)
        # bert_feature = self.drop_feature(bert_feature)  # 对 bert 后的特征表示做 dropout
        bert_feature = bert_feature.unsqueeze(2).expand([-1, -1, self.args.max_sequence_len, -1])
        bert_feature_T = bert_feature.transpose(1, 2)
        
        features = torch.cat([bert_feature, bert_feature_T], dim=3)
        
        # sim_matrix = torch.nn.functional.cosine_similarity(bert_feature, bert_feature_T, dim=3)
        # sim_matrix *= masks
        
        hidden = self.linear1(features)
        # hidden = self.drop_feature(hidden)
        hidden = self.norm1(hidden)
        hidden = self.gelu(hidden)

        logits = self.cls_linear(hidden)
        logits1 = self.cls_linear1(hidden)
        
        masks0 = masks.unsqueeze(3).expand([-1, -1, -1, self.args.class_num])#.shape
        masks1 = masks.unsqueeze(3).expand([-1, -1, -1, 2])#.shape
        
        logits = masks0 * logits
        logits1 = masks1 * logits1
        
        return logits, logits1, _#sim_matrix

In [ ]:
# Loss Func
class FocalLoss(torch.nn.Module):
    def __init__(self, weight, alpha=0.99, gamma=4., size_average=True, ignore_index=-1):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.size_average = size_average
        self.weight = weight

    def forward(self, inputs, targets):
        # F.cross_entropy(x,y)工作过程就是(Log_Softmax+NllLoss)：①对x做softmax,使其满足归一化要求，结果记为x_soft;②对x_soft做对数运算
        # 并取相反数，记为x_soft_log;③对y进行one-hot编码，编码后与x_soft_log进行点乘，只有元素为1的位置有值而且乘的是1，
        # 所以点乘后结果还是x_soft_log
        # 总之，F.cross_entropy(x,y)对应的数学公式就是CE(pt)=-1*log(pt)
        # inputs = inputs.long()
        # targets = targets.long()

        # print(inputs.dtype)
        # inputs = inputs.float()

        ce_loss = F.cross_entropy(inputs, targets, weight=self.weight, reduction='none', ignore_index=self.ignore_index)
        pt = torch.exp(-ce_loss)  # pt是预测该类别的概率，要明白F.cross_entropy工作过程就能够理解
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        if self.size_average:
            return focal_loss.mean()
        else:
            return focal_loss.sum()

# label = ['N', 'CTD', 'POS', 'NEU', 'NEG']
weight = torch.tensor([1.0, 4.0, 4.0, 4.0, 4.0]).float().cuda()
f_loss = FocalLoss(weight, ignore_index=-1)#.forwardf_loss(preds,labels)

weight1 = torch.tensor([1.0, 4.0]).float().cuda()
f_loss1 = FocalLoss(weight1, ignore_index=-1)
# loss = FocalLoss(ignore_index=-1).forward(preds,labels)

# Metric

In [ ]:
class Metric():
    '''评价指标 precision recall f1'''
    def __init__(self, args, stop_words, tokenized, ids, predictions, goldens, sen_lengths, tokens_ranges, ignore_index=-1):
        # metric = Metric(args, all_preds, all_labels, all_lengths, all_sens_lengths, all_token_ranges, ignore_index=-1)
        # print([i.sum() for i in predictions], [i.sum() for i in goldens])
        
        _g = np.array(goldens)
        _g[_g==-1] = 0
        
        print('sum_pred: ', np.array(predictions).sum(), ' sum_gt: ', _g.sum())
        self.args = args
        self.predictions = predictions
        self.goldens = goldens
        # self.bert_lengths = bert_lengths
        self.sen_lengths = sen_lengths
        self.tokens_ranges = tokens_ranges
        self.ignore_index = ignore_index
        self.data_num = len(self.predictions)
        self.ids = ids
        self.stop_words = stop_words
        self.tokenized = tokenized

    def get_spans(self, tags, length, token_range, type):
        spans = []
        start = -1
        for i in range(length):
            l, r = token_range[i]
            if tags[l][l] == self.ignore_index:
                continue
            elif tags[l][l] == type:
                if start == -1:
                    start = i
            elif tags[l][l] != type:
                if start != -1:
                    spans.append([start, i - 1])
                    start = -1
        if start != -1:
            spans.append([start, length - 1])
        return spans

    def find_triplet_golden(self, tag):
        triplets = []
        for row in range(1, tag.shape[0]-1):
            for col in range(1, tag.shape[1]-1):
                if row==col:
                    pass
                elif tag[row][col] in args.sentiment2id.values():
                    sentiment = int(tag[row][col])
                    al, pl = row, col
                    ar = al
                    pr = pl
                    while tag[ar+1][pr] == 1:
                        ar += 1
                    while tag[ar][pr+1] == 1:
                        pr += 1

                    triplets.append([al, ar, pl, pr, sentiment])

                # print(triplets)
                # [[1, 3, 6, 6, 5]]
                # [[1, 3, 6, 6, 5], [9, 11, 13, 13, 5]]
                # [[1, 3, 6, 6, 5], [9, 11, 13, 13, 5], [16, 16, 15, 15, 5]]
                
        return triplets
    
    def find_triplet(self, tag, ws, tokenized):
        triplets = []
        for row in range(1, tag.shape[0]-1):
            for col in range(1, tag.shape[1]-1):
                if row==col:
                    pass
                elif tag[row][col] in args.sentiment2id.values():
                    sentiment = int(tag[row][col])
                    al, pl = row, col
                    ar = al
                    pr = pl
                    while tag[ar+1][pr] == 1:
                        ar += 1
                    while tag[ar][pr+1] == 1:
                        pr += 1
                    
                    '''filting the illegal preds'''
                    
                    condition1 = al in np.array(ws)[:, 0] and ar in np.array(ws)[:, 1] and pl in np.array(ws)[:, 0] and pr in np.array(ws)[:, 1]
                    condition2 = True
                    for ii in range(al, ar+1):
                        for jj in range(pl, pr+1):
                            if ii == jj:
                                condition2 = False
                    
                    condition3 = True
                    for tk in tokenized[al: ar+1]:
                        # print(tk)
                        if tk in self.stop_words:
                            condition3 = False
                            break
                        
                    condition4 = True
                    for tk in tokenized[pl: pr+1]:
                        # print(tk)
                        if tk in self.stop_words:
                            condition4 = False
                            break

                    conditions = condition1 and condition2 and condition3 and condition4                        
                    # conditions = condition1 and condition2

                    if conditions:
                        triplets.append([al, ar, pl, pr, sentiment])

                # print(triplets)
                # [[1, 3, 6, 6, 5]]
                # [[1, 3, 6, 6, 5], [9, 11, 13, 13, 5]]
                # [[1, 3, 6, 6, 5], [9, 11, 13, 13, 5], [16, 16, 15, 15, 5]]
                
        return triplets
    
    def get_sets(self):
        assert len(self.predictions) == len(self.goldens)
        golden_set = set()
        predicted_set = set()
        for i in range(self.data_num):
            # golden_aspect_spans = self.get_spans(self.goldens[i], self.sen_lengths[i], self.tokens_ranges[i], 1)
            # golden_opinion_spans = self.get_spans(self.goldens[i], self.sen_lengths[i], self.tokens_ranges[i], 2)
            id = self.ids[i]
            golden_tuples = self.find_triplet_golden(np.array(self.goldens[i]))
            # golden_tuples: triplets.append([al, ar, pl, pr, sentiment])
            for golden_tuple in golden_tuples:
                golden_set.add(id + '-' + '-'.join(map(str, golden_tuple)))  # 从前到后把得到的三元组纳入总集合
                # golden_set: ('0-{al}-{ar}-{pl}-{pr}-{sentiment}', '1-{al}-{ar}-{pl}-{pr}-{sentiment}', '2-{al}-{ar}-{pl}-{pr}-{sentiment}')

            # predicted_aspect_spans = self.get_spans(self.predictions[i], self.sen_lengths[i], self.tokens_ranges[i], 1)
            # predicted_opinion_spans = self.get_spans(self.predictions[i], self.sen_lengths[i], self.tokens_ranges[i], 2)
            # if self.args.task == 'pair':
            #     predicted_tuples = self.find_pair(self.predictions[i], predicted_aspect_spans, predicted_opinion_spans, self.tokens_ranges[i])
            # elif self.args.task == 'triplet':
                
            tag = np.array(self.predictions[i])

            tag[0][:] = -1
            tag[-1][:] = -1
            tag[:, 0] = -1
            tag[:, -1] = -1

            predicted_triplets = self.find_triplet(tag, self.tokens_ranges[i], self.tokenized[i])  # , predicted_aspect_spans, predicted_opinion_spans, self.tokens_ranges[i]
            for pair in predicted_triplets:
                predicted_set.add(id + '-' + '-'.join(map(str, pair)))
        return predicted_set, golden_set

    def score_triplets(self, predicted_set, golden_set):
        correct_num = len(golden_set & predicted_set)
        precision = correct_num / len(predicted_set) if len(predicted_set) > 0 else 0
        recall = correct_num / len(golden_set) if len(golden_set) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        logging('Triplet\tP:{:.5f}\tR:{:.5f}\tF1:{:.5f}'.format(precision, recall, f1))
        return precision, recall, f1

    def score_pairs(self, predicted_set, golden_set):
        predicted_set = set(['-'.join(i.split('-')[0: 5]) for i in predicted_set])
        golden_set = set(['-'.join(i.split('-')[0: 5]) for i in golden_set])
        
        correct_num = len(golden_set & predicted_set)
        precision = correct_num / len(predicted_set) if len(predicted_set) > 0 else 0
        recall = correct_num / len(golden_set) if len(golden_set) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        logging('Pair\tP:{:.5f}\tR:{:.5f}\tF1:{:.5f}'.format(precision, recall, f1))
        return precision, recall, f1
    
    def score_aspect(self, predicted_set, golden_set):
        predicted_set = set(['-'.join(i.split('-')[0: 3]) for i in predicted_set])
        golden_set = set(['-'.join(i.split('-')[0: 3]) for i in golden_set])

        correct_num = len(golden_set & predicted_set)
        precision = correct_num / len(predicted_set) if len(predicted_set) > 0 else 0
        recall = correct_num / len(golden_set) if len(golden_set) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        logging('Aspect\tP:{:.5f}\tR:{:.5f}\tF1:{:.5f}'.format(precision, recall, f1))
        return precision, recall, f1

    def score_opinion(self, predicted_set, golden_set):
        predicted_set = set([i.split('-')[0] + '-' + ('-'.join(i.split('-')[3: 5])) for i in predicted_set])
        golden_set = set([i.split('-')[0] + '-' + ('-'.join(i.split('-')[3: 5])) for i in golden_set])

        correct_num = len(golden_set & predicted_set)
        precision = correct_num / len(predicted_set) if len(predicted_set) > 0 else 0
        recall = correct_num / len(golden_set) if len(golden_set) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        logging('Opinion\tP:{:.5f}\tR:{:.5f}\tF1:{:.5f}'.format(precision, recall, f1))
        return precision, recall, f1

# Eval

In [ ]:
def get_triplets_set(id_, word_span, tagging_matrix_pred):
    valid_len = word_span[-1][-1] + 1
    st_list = [i[0] for i in word_span]
    end_list = [i[1] for i in word_span]
    triplets_pred = []
    for row in range(valid_len):
        for col in range(valid_len):
            if row == col:
                pass
            elif tagging_matrix_pred[row][col] in args.sentiment2id.values():
                sentiment = int(tagging_matrix_pred[row][col].detach().cpu())
                al, pl = row, col
                ar = al
                pr = pl
                while tagging_matrix_pred[ar+1][pr] == 1:
                    ar += 1
                while tagging_matrix_pred[ar][pr+1] == 1:
                    pr += 1
                    
                if (al in st_list) and (pl in st_list) and (ar in end_list) and (pr in end_list): # ensure the sanity
                    if min(al, ar) > max(pl, pr) or min(pl, pr) > max(al, ar):
                        triplets_pred.append([al, ar, pl, pr, sentiment])

    predicted_set = set()
    for tri in triplets_pred:
        predicted_set.add(str(id_) + '-' + '-'.join(map(str, tri)))
    return predicted_set


def eval(model, dataset, stop_words):
    model.eval()
    with torch.no_grad():
        all_ids = []
        all_preds = []
        all_labels = []
        # all_lengths = []
        all_sens_lengths = []
        all_token_ranges = []
        all_tokenized = []

        for i in range(dataset.batch_count):
            sentence_ids, tokens, masks, token_ranges, tags, tokenized = dataset.get_batch(i)
            # sentence_ids, bert_tokens, masks, word_spans, tagging_matrices = trainset.get_batch(i)
            preds, _, _ = model(tokens, masks) #1
            preds = torch.argmax(preds, dim=3) #2
            all_preds.append(preds) #3
            all_labels.append(tags) #4
            # all_lengths.append(lengths) #5
            sens_lens = [len(token_range) for token_range in token_ranges]
            all_sens_lengths.extend(sens_lens) #6
            all_token_ranges.extend(token_ranges) #7
            all_ids.extend(sentence_ids) #8
            all_tokenized.extend(tokenized)

        all_preds = torch.cat(all_preds, dim=0).cpu().tolist()
        all_labels = torch.cat(all_labels, dim=0).cpu().tolist()
        # all_lengths = torch.cat(all_lengths, dim=0).cpu().tolist()
        
        # 引入 metric 计算评价指标
        metric = Metric(args, stop_words, all_tokenized, all_ids, all_preds, all_labels, all_sens_lengths, all_token_ranges, ignore_index=-1)
        predicted_set, golden_set = metric.get_sets()
        
        
        aspect_results = metric.score_aspect(predicted_set, golden_set)
        opinion_results = metric.score_opinion(predicted_set, golden_set)
        pair_results = metric.score_pairs(predicted_set, golden_set)
        
        precision, recall, f1 = metric.score_triplets(predicted_set, golden_set)
        
        
        logging('Aspect term\tP:{:.5f}\tR:{:.5f}\tF1:{:.5f}'.format(aspect_results[0], aspect_results[1],
                                                                  aspect_results[2]))
        logging('Opinion term\tP:{:.5f}\tR:{:.5f}\tF1:{:.5f}'.format(opinion_results[0], opinion_results[1], 
                                                                     opinion_results[2]))
        logging(args.task + '\tP:{:.5f}\tR:{:.5f}\tF1:{:.5f}\n'.format(precision, recall, f1))

    model.train()
    return precision, recall, f1


# Train

In [ ]:
from string import punctuation
stop_words = []#set()

for i in punctuation:
    stop_words.append('Ġ' + i)
stop_words   

['Ġ!',
 'Ġ"',
 'Ġ#',
 'Ġ$',
 'Ġ%',
 'Ġ&',
 "Ġ'",
 'Ġ(',
 'Ġ)',
 'Ġ*',
 'Ġ+',
 'Ġ,',
 'Ġ-',
 'Ġ.',
 'Ġ/',
 'Ġ:',
 'Ġ;',
 'Ġ<',
 'Ġ=',
 'Ġ>',
 'Ġ?',
 'Ġ@',
 'Ġ[',
 'Ġ\\',
 'Ġ]',
 'Ġ^',
 'Ġ_',
 'Ġ`',
 'Ġ{',
 'Ġ|',
 'Ġ}',
 'Ġ~']

In [ ]:
# Load Model
model = MultiInferBert(args).to(args.device)

optimizer = torch.optim.Adam([
                {'params': model.bert.parameters(), 'lr': 1e-5},
                {'params': model.linear1.parameters(), 'lr': 1e-2},
                {'params': model.cls_linear.parameters(), 'lr': 1e-3},
                {'params': model.cls_linear1.parameters(), 'lr': 1e-3}
            ], lr=1e-3)#SGD, momentum=0.9


lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200, 600, 1000], gamma=0.5, verbose=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Adjusting learning rate of group 0 to 1.0000e-05.
Adjusting learning rate of group 1 to 1.0000e-02.
Adjusting learning rate of group 2 to 1.0000e-03.
Adjusting learning rate of group 3 to 1.0000e-03.


In [ ]:
best_joint_f1 = 0
best_joint_f1_test = 0
best_joint_epoch = 0
best_joint_epoch_test = 0

model.train()

for i in range(args.epochs):
    logging('\n\nEpoch:{}'.format(i+1))
    epoch_sum_loss = []
    for j in trange(trainset.batch_count):
        sentence_ids, bert_tokens, masks, word_spans, tagging_matrices, tokenized = trainset.get_batch(j)
        
        # mask_cl = ((torch.tril(torch.ones(args.max_sequence_len, args.max_sequence_len)) - 1) * (-1)).unsqueeze(0).expand(len(sentence_ids), -1, -1).to(args.device)

        # bert_tokens = bert_tokens.half().to(args.device)
        # masks = masks.half().to(args.device)

        logits, logits1, sim_matrix = model(bert_tokens, masks)

        logits_flatten = logits.reshape([-1, logits.shape[3]])
        tagging_matrices_flatten = tagging_matrices.reshape([-1])
        
        loss0 = f_loss(logits_flatten, tagging_matrices_flatten)#, logits1, sim_matrix

        tags1 = tagging_matrices.clone()#[tags>0]
        tags1[tags1>0] = 1
        logits1_flatten = logits1.reshape([-1, logits1.shape[3]])
        tags1_flatten = tags1.reshape([-1]).to(args.device)
        loss1 = f_loss1(logits1_flatten.float(), tags1_flatten)
        
        # tags2 = tags1.clone()#[tags>0]
        # '''transpose so that only a triangle is used'''
        # tags2 = tags2 + tags2.transpose(1, 2)
        
        # tags2 = 1.5 * tags2**2 + 0.5 * tags2 - 1
        
        # loss_cl = - (tags2 * sim_matrix * mask_cl).sum()
        
        loss = loss0 + loss1#.half()# loss0 + loss1 #+ 1e-6 * loss_cl # + 1e-12 * loss_cl
        epoch_sum_loss.append(loss)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        writer.add_scalar('train loss', loss, i*trainset.batch_count+j+1)
        writer.add_scalar('train loss0', loss0, i*trainset.batch_count+j+1)
        writer.add_scalar('train loss1', loss1, i*trainset.batch_count+j+1)
        writer.add_scalar('lr', optimizer.param_groups[0]['lr'], i*trainset.batch_count+j+1)
        # writer.add_scalar('lr1', optimizer.param_groups[1]['lr'], i*trainset.batch_count+j+1)
        # writer.add_scalar('lr2', optimizer.param_groups[2]['lr'], i*trainset.batch_count+j+1)
        # writer.add_scalar('lr3', optimizer.param_groups[3]['lr'], i*trainset.batch_count+j+1)
        
    epoch_avg_loss = sum(epoch_sum_loss) / len(epoch_sum_loss)
    

    logging('{}\tAvg loss: {:.10f}'.format(str(datetime.datetime.now()), epoch_avg_loss))
        
    joint_precision, joint_recall, joint_f1 = eval(model, devset, stop_words)
    joint_precision_test, joint_recall_test, joint_f1_test = eval(model, testset, stop_words)
    
    if joint_f1 >= best_joint_f1:
        best_joint_f1 = joint_f1
        best_joint_epoch = i
        if joint_f1_test > best_joint_f1_test:
            model_path = args.model_dir + args.data_version + args.dataset + str(joint_f1_test) + 'epoch' + str(i) + '.pt'
            torch.save(model, model_path)
            best_joint_f1_test = joint_f1_test
            best_joint_epoch_test = i

    writer.add_scalar('dev f1', joint_f1, i+1)
    writer.add_scalar('test f1', joint_f1_test, i+1)
    writer.add_scalar('dev precision', joint_precision, i+1)
    writer.add_scalar('test precision', joint_precision_test, i+1)
    writer.add_scalar('dev recall', joint_recall, i+1)
    writer.add_scalar('test recall', joint_recall_test, i+1)
    
    lr_scheduler.step()


    logging('best epoch: {}\tbest dev {} f1: {:.5f}'.format(best_joint_epoch+1, args.task, best_joint_f1))
    logging('best epoch: {}\tbest test {} f1: {:.5f}'.format(best_joint_epoch_test+1, args.task, best_joint_f1_test))


# 关闭TensorBoard写入器
writer.close()



Epoch:1


100%|██████████| 79/79 [00:11<00:00,  6.98it/s]


2023-10-09 15:43:44.773563	Avg loss: 0.0068075219
sum_pred:  634  sum_gt:  2769
Aspect	P:0.00000	R:0.00000	F1:0.00000
Opinion	P:0.00000	R:0.00000	F1:0.00000
Pair	P:0.00000	R:0.00000	F1:0.00000
Triplet	P:0.00000	R:0.00000	F1:0.00000
Aspect term	P:0.00000	R:0.00000	F1:0.00000
Opinion term	P:0.00000	R:0.00000	F1:0.00000
triplet	P:0.00000	R:0.00000	F1:0.00000

sum_pred:  1625  sum_gt:  5440
Aspect	P:0.00000	R:0.00000	F1:0.00000
Opinion	P:0.00000	R:0.00000	F1:0.00000
Pair	P:0.00000	R:0.00000	F1:0.00000
Triplet	P:0.00000	R:0.00000	F1:0.00000
Aspect term	P:0.00000	R:0.00000	F1:0.00000
Opinion term	P:0.00000	R:0.00000	F1:0.00000
triplet	P:0.00000	R:0.00000	F1:0.00000

Adjusting learning rate of group 0 to 1.0000e-05.
Adjusting learning rate of group 1 to 1.0000e-02.
Adjusting learning rate of group 2 to 1.0000e-03.
Adjusting learning rate of group 3 to 1.0000e-03.
best epoch: 1	best dev triplet f1: 0.00000
best epoch: 1	best test triplet f1: 0.00000


Epoch:2


100%|██████████| 79/79 [00:11<00:00,  7.03it/s]


2023-10-09 15:44:03.542323	Avg loss: 0.0037197543
sum_pred:  5415  sum_gt:  2769
Aspect	P:0.65979	R:0.56763	F1:0.61025
Opinion	P:0.72818	R:0.62661	F1:0.67359
Pair	P:0.39089	R:0.46805	F1:0.42601
Triplet	P:0.33281	R:0.39850	F1:0.36270
Aspect term	P:0.65979	R:0.56763	F1:0.61025
Opinion term	P:0.72818	R:0.62661	F1:0.67359
triplet	P:0.33281	R:0.39850	F1:0.36270

sum_pred:  12195  sum_gt:  5440
Aspect	P:0.70543	R:0.59091	F1:0.64311
Opinion	P:0.73458	R:0.65057	F1:0.69003
Pair	P:0.36716	R:0.48740	F1:0.41882
Triplet	P:0.31766	R:0.42169	F1:0.36235
Aspect term	P:0.70543	R:0.59091	F1:0.64311
Opinion term	P:0.73458	R:0.65057	F1:0.69003
triplet	P:0.31766	R:0.42169	F1:0.36235

Adjusting learning rate of group 0 to 1.0000e-05.
Adjusting learning rate of group 1 to 1.0000e-02.
Adjusting learning rate of group 2 to 1.0000e-03.
Adjusting learning rate of group 3 to 1.0000e-03.
best epoch: 2	best dev triplet f1: 0.36270
best epoch: 2	best test triplet f1: 0.36235


Epoch:3


...

2023-10-09 23:06:42.347422	Avg loss: 0.0000094738
sum_pred:  2857  sum_gt:  2769
Aspect	P:0.83562	R:0.81153	F1:0.82340
Opinion	P:0.85746	R:0.83906	F1:0.84816
Pair	P:0.75047	R:0.75188	F1:0.75117
Triplet	P:0.69418	R:0.69549	F1:0.69484
Aspect term	P:0.83562	R:0.81153	F1:0.82340
Opinion term	P:0.85746	R:0.83906	F1:0.84816
triplet	P:0.69418	R:0.69549	F1:0.69484

sum_pred:  5437  sum_gt:  5440
Aspect	P:0.85864	R:0.85195	F1:0.85528
Opinion	P:0.86667	R:0.87548	F1:0.87105
Pair	P:0.78051	R:0.79847	F1:0.78939
Triplet	P:0.74197	R:0.75904	F1:0.75041
Aspect term	P:0.85864	R:0.85195	F1:0.85528
Opinion term	P:0.86667	R:0.87548	F1:0.87105
triplet	P:0.74197	R:0.75904	F1:0.75041

Adjusting learning rate of group 0 to 1.2500e-06.
Adjusting learning rate of group 1 to 1.2500e-03.
Adjusting learning rate of group 2 to 1.2500e-04.
Adjusting learning rate of group 3 to 1.2500e-04.
best epoch: 999	best dev triplet f1: 0.71251
best epoch: 999	best test triplet f1: 0.75998


Epoch:1415


100%|██████████| 79/79 [00:11<00:00,  6.99it/s]


2023-10-09 23:07:01.486417	Avg loss: 0.0000094728
sum_pred:  2844  sum_gt:  2769
Aspect	P:0.84138	R:0.81153	F1:0.82619
Opinion	P:0.85714	R:0.83691	F1:0.84691
Pair	P:0.75614	R:0.75188	F1:0.75401
Triplet	P:0.69943	R:0.69549	F1:0.69746
Aspect term	P:0.84138	R:0.81153	F1:0.82619
Opinion term	P:0.85714	R:0.83691	F1:0.84691
triplet	P:0.69943	R:0.69549	F1:0.69746

sum_pred:  5425  sum_gt:  5440
Aspect	P:0.85976	R:0.85195	F1:0.85584
Opinion	P:0.86667	R:0.87548	F1:0.87105
Pair	P:0.78051	R:0.79847	F1:0.78939
Triplet	P:0.74197	R:0.75904	F1:0.75041
Aspect term	P:0.85976	R:0.85195	F1:0.85584
Opinion term	P:0.86667	R:0.87548	F1:0.87105
triplet	P:0.74197	R:0.75904	F1:0.75041

Adjusting learning rate of group 0 to 1.2500e-06.
Adjusting learning rate of group 1 to 1.2500e-03.
Adjusting learning rate of group 2 to 1.2500e-04.
Adjusting learning rate of group 3 to 1.2500e-04.
best epoch: 999	best dev triplet f1: 0.71251
best epoch: 999	best test triplet f1: 0.75998


Epoch:1416


100%|██████████| 79/79 [00:11<00:00,  6.89it/s]


2023-10-09 23:07:20.830429	Avg loss: 0.0000094658
sum_pred:  2842  sum_gt:  2769
Aspect	P:0.84138	R:0.81153	F1:0.82619
Opinion	P:0.85903	R:0.83691	F1:0.84783
Pair	P:0.75758	R:0.75188	F1:0.75472
Triplet	P:0.70076	R:0.69549	F1:0.69811
Aspect term	P:0.84138	R:0.81153	F1:0.82619
Opinion term	P:0.85903	R:0.83691	F1:0.84783
triplet	P:0.70076	R:0.69549	F1:0.69811

sum_pred:  5417  sum_gt:  5440


In [ ]:
# !pip install thop

In [ ]:
# from thop import profile
 
# # input = torch.randn(1, 3, 224, 224)
# flops, params = profile(model, inputs=(bert_tokens, masks))
# print(flops)
# print(params)


# Translation

In [ ]:
model = MultiInferBert(args).to(args.device)
model_path = '.\savemodel\D1res140.7335824879871863epoch154.pt'
model.load_state_dict(torch.load(model_path).state_dict())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
dataset = trainset
model.eval()
with torch.no_grad():
    all_ids = []
    all_preds = []
    all_labels = []
    # all_lengths = []
    all_sens_lengths = []
    all_token_ranges = []
    all_tokenized = []
    for i in range(dataset.batch_count):
        sentence_ids, tokens, masks, token_ranges, tags, tokenized = dataset.get_batch(i)
        tokens = tokens.to(args.device)
        masks = masks.to(args.device)
        tags = tags.to(args.device)

        preds, _, _ = model(tokens, masks) #1
        preds = torch.argmax(preds, dim=3) #2
        all_preds.append(preds) #3
        all_labels.append(tags) #4
        # all_lengths.append(lengths) #5
        sens_lens = [len(token_range) for token_range in token_ranges]
        all_sens_lengths.extend(sens_lens) #6
        all_token_ranges.extend(token_ranges) #7
        all_ids.extend(sentence_ids) #8
        all_tokenized.extend(tokenized)

    all_preds = torch.cat(all_preds, dim=0).cpu().tolist()
    all_labels = torch.cat(all_labels, dim=0).cpu().tolist()
    
    # 引入 metric 计算评价指标
    metric = Metric(args, stop_words, all_tokenized, all_ids, all_preds, all_labels, all_sens_lengths, all_token_ranges, ignore_index=-1)
    # Metric(args, all_ids, all_preds, all_labels, all_sens_lengths, all_token_ranges, ignore_index=-1)
    predicted_set, golden_set = metric.get_sets()

sum_pred:  11494  sum_gt:  11515


In [ ]:
predicted_set, golden_set

({'2215-20-20-23-23-4',
  '272-6-8-5-5-4',
  '139-11-13-5-5-4',
  '2823-18-18-2-2-4',
  '2120-15-16-18-18-4',
  '3230-18-18-22-23-4',
  '1894-12-12-11-11-4',
  '466-13-13-15-15-4',
  '3625-2-2-1-1-4',
  '1827-6-6-3-3-4',
  '2741-17-17-16-16-4',
  '1666-8-8-5-6-2',
  '2976-4-6-8-10-4',
  '378-7-8-6-6-4',
  '1015-2-2-1-1-4',
  '1585-7-8-2-5-4',
  '257-5-8-14-14-3',
  '479-1-4-6-9-4',
  '3048-11-11-7-7-2',
  '1874-8-8-10-11-2',
  '2125-6-8-11-11-4',
  '3202-45-47-9-9-4',
  '1690-17-17-3-3-2',
  '1078-1-2-5-5-3',
  '2774-12-13-6-6-2',
  '2436-12-13-9-11-4',
  '2104-25-27-21-24-3',
  '89-23-24-28-29-4',
  '1268-18-19-17-17-2',
  '2983-4-4-7-7-4',
  '2334-4-7-2-2-3',
  '2879-1-1-3-3-4',
  '1797-21-21-20-20-2',
  '2008-1-6-14-14-4',
  '3482-14-16-12-12-4',
  '3040-5-5-11-11-4',
  '898-23-23-13-13-2',
  '978-1-1-4-5-4',
  '189-27-27-25-26-2',
  '2993-2-2-1-1-4',
  '140-6-22-3-3-4',
  '3423-3-4-8-10-4',
  '1322-4-8-3-3-4',
  '2006-1-1-6-6-4',
  '2413-13-13-1-1-2',
  '1714-7-17-24-24-4',
  '3658

In [ ]:
# for i in train_sentence_packs:
import pandas as pd
df_train = pd.DataFrame(train_sentence_packs)#.to_csv('train_sentence_packs.csv', index=False)
df_train.index = df_train['id']
df_train

,id,sentence,postag,head,deprel,triples
id,,,,,,
314,314,All I can say is $ 2 pints during happy hour a...,"[DT, PRP, MD, VB, VBZ, $, CD, NNS, IN, JJ, NN,...","[6, 4, 4, 1, 6, 0, 8, 6, 11, 11, 6, 14, 14, 6,...","[nsubj, nsubj, aux, acl:relcl, cop, root, comp...","[{'uid': '314-0', 'target_tags': 'All\O I\O ca..."
339,339,The sushi was awful !,"[DT, NN, VBD, JJ, .]","[2, 4, 4, 0, 4]","[det, nsubj, cop, root, punct]","[{'uid': '339-0', 'target_tags': 'The\O sushi\..."
796,796,"The food was just OK , at least for what food ...","[DT, NN, VBD, RB, JJ, ,, RB, RBS, IN, WDT, NN,...","[2, 5, 5, 5, 0, 5, 13, 7, 13, 11, 13, 13, 5, 5]","[det, nsubj, cop, advmod, root, punct, advmod,...","[{'uid': '796-0', 'target_tags': 'The\O food\B..."
1534,1534,I did n't expect to like Nosh as much as I did...,"[PRP, VBD, RB, VB, TO, VB, NNP, RB, RB, IN, PR...","[4, 4, 4, 0, 6, 4, 6, 9, 6, 12, 12, 8, 23, 23,...","[nsubj, aux, advmod, root, mark, xcomp, obj, a...","[{'uid': '1534-0', 'target_tags': 'I\O did\O n..."
1061,1061,Service is fast and friendly .,"[NN, VBZ, JJ, CC, JJ, .]","[3, 3, 0, 5, 3, 3]","[nsubj, cop, root, cc, conj, punct]","[{'uid': '1061-0', 'target_tags': 'Service\B i..."
...,...,...,...,...,...,...
1294,1294,They smell like they stuff them with old canne...,"[PRP, VBP, IN, PRP, VBP, PRP, IN, JJ, VBN, NNS...","[2, 0, 5, 5, 2, 5, 10, 10, 10, 5, 15, 15, 15, ...","[nsubj, root, mark, nsubj, advcl, obj, case, a...","[{'uid': '1294-0', 'target_tags': 'They\O smel..."
913,913,Traditional French decour was pleasant though ...,"[JJ, JJ, NN, VBD, JJ, IN, DT, NN, VBD, RB, JJ,...","[3, 3, 5, 5, 0, 11, 8, 11, 11, 11, 5, 5, 14, 1...","[amod, amod, nsubj, cop, root, mark, det, nsub...","[{'uid': '913-0', 'target_tags': 'Traditional\..."
2170,2170,The staff is no nonsense .,"[DT, NN, VBZ, DT, NN, .]","[2, 5, 5, 5, 0, 5]","[det, nsubj, cop, det, root, punct]","[{'uid': '2170-0', 'target_tags': 'The\O staff..."


In [ ]:
for i in predicted_set:
    print(i)
    # id = 
    sentence_id = i.split('-')[0]
    sentence = df_train.loc[sentence_id, 'sentence']
    sentence_split = sentence.split(' ')
    al = int(i.split('-')[1])
    ar = int(i.split('-')[2])
    pl = int(i.split('-')[3])
    pr = int(i.split('-')[4])

    # sentence = df_train.loc[sentence_id, 'sentence']
    aspect = sentence_split[al: ar+1]
    opinion = sentence_split[pl: pr+1]

    print(sentence, aspect, opinion)

    # df_train[i.split('-')[0]]
    #.loc[int(i.split('-')[0]), 'triplet'] = '-'.join(i.split('-')[1:])
    # break

2215-20-20-23-23-4
The ambience was so fun , and the prices were great , on top of the fact that the food was really tasty . ['was'] ['.']
272-6-8-5-5-4
Pair you food with the excellent beers on tap or their well priced wine list . ['beers', 'on', 'tap'] ['excellent']
139-11-13-5-5-4
The food here does a great service to the name ( Cantonese that is ... ) . ['Cantonese', 'that', 'is'] ['great']
2823-18-18-2-2-4
I highly recommend Cafe St. Bart 's for their food , the ambience and wonderful service . [] ['recommend']
2120-15-16-18-18-4
We recently decided to try this location , and to our delight , they have outdoor seating , perfect since I had my yorkie with me . ['outdoor', 'seating'] ['perfect']
3230-18-18-22-23-4
The seafood is amazing , there 's a good wine list , and the ever-changing menu always offers some great surprises . ['some'] []
1894-12-12-11-11-4
The place is a BISTRO which means : simple dishes and wine served efficiently in a bustling atmosphere . ['served'] ['wine']


In [ ]:
eval(model, testset)

sum_pred:  5859  sum_gt:  5440
Triplet	P:0.71562	R:0.75246	F1:0.73358


(0.715625, 0.7524644030668127, 0.7335824879871863)

In [ ]:
dataset = trainset
model.eval()
with torch.no_grad():
    all_ids = []
    all_preds = []
    all_labels = []
    # all_lengths = []
    all_sens_lengths = []
    all_token_ranges = []
    for i in range(dataset.batch_count):
        sentence_ids, tokens, masks, token_ranges, tags = dataset.get_batch(i)
        tokens = tokens.to(args.device)
        masks = masks.to(args.device)
        tags = tags.to(args.device)

        preds, _, _ = model(tokens, masks) #1
        preds = torch.argmax(preds, dim=3) #2
        all_preds.append(preds) #3
        all_labels.append(tags) #4
        # all_lengths.append(lengths) #5
        sens_lens = [len(token_range) for token_range in token_ranges]
        all_sens_lengths.extend(sens_lens) #6
        all_token_ranges.extend(token_ranges) #7
        all_ids.extend(sentence_ids) #8

    all_preds = torch.cat(all_preds, dim=0).cpu().tolist()
    all_labels = torch.cat(all_labels, dim=0).cpu().tolist()
    
    # 引入 metric 计算评价指标
    metric = Metric(args, all_preds, all_labels, all_sens_lengths, all_token_ranges, ignore_index=-1)
    predicted_set, golden_set = metric.get_sets()
    
    # precision, recall, f1 = metric.score_triplets(predicted_set, golden_set)

# model.train()

sum_pred:  11505  sum_gt:  11515


In [ ]:
predicted_set, golden_set

({'198-1-1-3-3-4',
  '299-2-5-1-1-4',
  '734-23-23-13-13-2',
  '1127-1-1-4-4-4',
  '375-7-7-5-6-4',
  '619-1-1-3-3-2',
  '317-17-17-16-16-2',
  '1116-1-1-7-7-2',
  '787-4-4-28-28-4',
  '595-1-2-10-10-4',
  '570-6-7-5-5-4',
  '1078-12-16-9-11-4',
  '379-5-6-16-16-4',
  '160-9-9-8-8-4',
  '115-1-2-4-4-2',
  '1168-1-2-6-8-2',
  '499-18-18-17-17-4',
  '1081-11-11-9-9-4',
  '908-2-2-1-1-4',
  '1243-18-18-17-17-4',
  '1228-1-1-3-3-4',
  '1071-20-20-13-14-4',
  '508-6-7-1-1-4',
  '368-5-5-4-4-2',
  '333-10-10-9-9-4',
  '922-15-16-26-26-4',
  '1155-13-13-8-12-2',
  '1067-10-10-16-16-4',
  '37-4-5-7-7-4',
  '972-18-20-17-17-4',
  '533-9-10-7-7-2',
  '759-4-4-6-7-4',
  '837-6-6-2-2-3',
  '471-18-18-14-14-2',
  '671-15-15-14-14-4',
  '1051-8-8-7-7-4',
  '504-3-3-5-5-4',
  '793-22-23-21-21-4',
  '896-8-8-4-5-3',
  '131-14-14-17-17-4',
  '390-25-25-23-23-2',
  '1136-9-10-17-17-4',
  '1223-1-1-4-4-4',
  '986-1-1-3-3-2',
  '292-9-9-11-11-3',
  '695-5-5-10-10-2',
  '738-4-9-1-1-4',
  '736-24-27-9-9-4'

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for name, param in refine.named_parameters():
    print(name, param.shape)

bert.embeddings.word_embeddings.weight torch.Size([50265, 768])
bert.embeddings.position_embeddings.weight torch.Size([514, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([1, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [ ]:
[
    [int(predicted_dict[i][0]), int(predicted_dict[i][1])]
    if i in predicted_dict.keys() 
    else [] 
    for i in sentence_ids
    ]

[
    [int(predicted_dict[i][2]), int(predicted_dict[i][3])]
    if i in predicted_dict.keys() 
    else [] 
    for i in sentence_ids
    ]

NameError: name 'predicted_dict' is not defined

In [ ]:
torch.zeros(16, 100)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
torch.eq(senti_preds.argmax(dim=1), senti_gt)
senti_preds.argmax(dim=1), senti_gt

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
 tensor([3, 1, 0, 1, 0, 1, 3, 0, 0, 1, 3, 3, 3, 0, 0, 0], device='cuda:0'))

In [ ]:
# best_joint_f1 = 0
# best_joint_f1_test = 0
# best_joint_epoch = 0
# best_joint_epoch_test = 0
import torch.nn.functional as F

class Refine(torch.nn.Module):
    def __init__(self):
        super(Refine, self).__init__()
        self.bert = RobertaModel.from_pretrained("roberta-base")
        self.linear1 = torch.nn.Linear(768*100, 768)
        self.linear2 = torch.nn.Linear(768, 4)
        self.relu = torch.nn.ReLU()

    def forward(self, tokens, masks):
        # tokens = [101] + tokens[0].tolist() + [102]
        # masks
        # x = self.bert(x)[0][:, 0, :]
        x, _ = self.bert(tokens, masks, return_dict=False)
        # print(x.shape)
        x = x.reshape([-1, 768*100])
        # print(x.shape)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = F.softmax(x, dim=1)
        return x

refine = Refine().to(args.device)


optimizer = torch.optim.Adam([
                {'params': refine.bert.parameters(), 'lr': 0},
                {'params': refine.linear1.parameters(), 'lr': 2e-7},
                {'params': refine.linear2.parameters()}
            ], lr=1e-5)#SGD, momentum=0.9
    
    #         lr=1e-3)#SGD, momentum=0.9


lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80, 200, 400, 900], gamma=0.1, verbose=True)


predicted_dict = {i.split('-')[0]: i.split('-')[1: ] for i in predicted_set}
golden_dict = {i.split('-')[0]: i.split('-')[1: ] for i in golden_set}



refine.train()
for i in range(args.epochs):
    t_num_correct = 0
    logging('\n\nEpoch:{}'.format(i+1))
    # epoch_sum_loss = []
    for j in trange(trainset.batch_count):
        sentence_ids, tokens, masks, word_spans, tagging_matrices = trainset.get_batch(j)
        # proposal = predicted_dict[i]

        senti_gt = torch.tensor([int(golden_dict[i][-1]) if i in golden_dict.keys() else 1 for i in sentence_ids]) - 1
        senti_gt = senti_gt.to(args.device)

        senti_preds = refine(tokens, masks)
        # print(senti_preds, senti_gt)

        loss = F.cross_entropy(senti_preds, senti_gt)
        # 判断相等标签的个数
        num_correct = torch.sum(torch.eq(senti_preds.argmax(dim=1), senti_gt))
        t_num_correct += num_correct.item()

        # 打印结果
        # print(num_correct.item())
                
    #     # mask_cl = ((torch.tril(torch.ones(args.max_sequence_len, args.max_sequence_len)) - 1) * (-1)).unsqueeze(0).expand(len(sentence_ids), -1, -1).to(args.device)

    #     # bert_tokens = bert_tokens.half().to(args.device)
    #     # masks = masks.half().to(args.device)

    #     logits, logits1, sim_matrix = model(bert_tokens, masks)

    #     logits_flatten = logits.reshape([-1, logits.shape[3]])
    #     tagging_matrices_flatten = tagging_matrices.reshape([-1])
        
    #     loss0 = f_loss(logits_flatten, tagging_matrices_flatten)#, logits1, sim_matrix

    #     tags1 = tagging_matrices.clone()#[tags>0]
    #     tags1[tags1>0] = 1
    #     logits1_flatten = logits1.reshape([-1, logits1.shape[3]])
    #     tags1_flatten = tags1.reshape([-1]).to(args.device)
    #     loss1 = f_loss1(logits1_flatten.float(), tags1_flatten)
        
    #     # tags2 = tags1.clone()#[tags>0]
    #     # '''transpose so that only a triangle is used'''
    #     # tags2 = tags2 + tags2.transpose(1, 2)
        
    #     # tags2 = 1.5 * tags2**2 + 0.5 * tags2 - 1
        
    #     # loss_cl = - (tags2 * sim_matrix * mask_cl).sum()
        
    #     loss = loss0 + loss1#.half()# loss0 + loss1 #+ 1e-6 * loss_cl # + 1e-12 * loss_cl
    #     epoch_sum_loss.append(loss)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # epoch_avg_loss = sum(epoch_sum_loss) / len(epoch_sum_loss)
    
    # logging('{}\tAvg loss: {:.10f}'.format(str(datetime.datetime.now()), epoch_avg_loss))
        
    # joint_precision, joint_recall, joint_f1 = eval(model, devset)
    # joint_precision_test, joint_recall_test, joint_f1_test = eval(model, testset)
    
    # if joint_f1 >= best_joint_f1:
    #     best_joint_f1 = joint_f1
    #     best_joint_epoch = i
    #     if joint_f1_test > best_joint_f1_test:
    #         model_path = args.model_dir + args.data_version + args.dataset + str(joint_f1_test) + 'epoch' + str(i) + '.pt'
    #         torch.save(model, model_path)
    #         best_joint_f1_test = joint_f1_test
    #         best_joint_epoch_test = i
    
    # logging('best epoch: {}\tbest dev {} f1: {:.5f}'.format(best_joint_epoch+1, args.task, best_joint_f1))
    # logging('best epoch: {}\tbest test {} f1: {:.5f}'.format(best_joint_epoch_test+1, args.task, best_joint_f1_test))
        # print(loss)    
    lr_scheduler.step()
    
    acc = t_num_correct / len(train_instances)
    print(acc)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Adjusting learning rate of group 0 to 0.0000e+00.
Adjusting learning rate of group 1 to 2.0000e-07.
Adjusting learning rate of group 2 to 1.0000e-05.


Epoch:1


100%|██████████| 79/79 [00:12<00:00,  6.51it/s]


Adjusting learning rate of group 0 to 0.0000e+00.
Adjusting learning rate of group 1 to 2.0000e-07.
Adjusting learning rate of group 2 to 1.0000e-05.
0.5901509134233519


Epoch:2


100%|██████████| 79/79 [00:12<00:00,  6.49it/s]


...

Adjusting learning rate of group 0 to 0.0000e+00.
Adjusting learning rate of group 1 to 2.0000e-07.
Adjusting learning rate of group 2 to 1.0000e-05.
0.6695790309769658


Epoch:5


100%|██████████| 79/79 [00:12<00:00,  6.49it/s]


Adjusting learning rate of group 0 to 0.0000e+00.
Adjusting learning rate of group 1 to 2.0000e-07.
Adjusting learning rate of group 2 to 1.0000e-05.
0.6695790309769658


Epoch:6


 65%|██████▍   | 51/79 [00:07<00:04,  6.40it/s]


KeyboardInterrupt: 

In [ ]:
len(train_instances)

1259

In [ ]:
predicted_set
predicted_dict = {i.split('-')[0]: i.split('-')[1: ] for i in predicted_set}
golden_dict = {i.split('-')[0]: i.split('-')[1: ] for i in golden_set}

for i in sentence_ids:
    if i in predicted_dict.keys():
        proposal = predicted_dict[i]
        print(proposal)
        print(tokens)

    



senti_gt = torch.tensor([int(golden_dict[i][-1]) if i in golden_dict.keys() else 1 for i in sentence_ids]) - 1
senti_gt = senti_gt.to(args.device)

senti_preds = refine(tokens, masks)

loss = F.cross_entropy(senti_preds, senti_gt)

        
    else:
        print('no proposal')
    # break

no proposal
['1', '1', '3', '3', '4']
tensor([[   20,  3984,   889,  ...,     0,     0,     0],
        [11087,     5,  2131,  ...,     0,     0,     0],
        [  152,   317,    16,  ...,     0,     0,     0],
        ...,
        [   20,   813,    16,  ...,     0,     0,     0],
        [ 2497,  5466,  2156,  ...,     0,     0,     0],
        [17335,  3634,   337,  ...,     0,     0,     0]], device='cuda:0')
['6', '6', '10', '10', '2']
tensor([[   20,  3984,   889,  ...,     0,     0,     0],
        [11087,     5,  2131,  ...,     0,     0,     0],
        [  152,   317,    16,  ...,     0,     0,     0],
        ...,
        [   20,   813,    16,  ...,     0,     0,     0],
        [ 2497,  5466,  2156,  ...,     0,     0,     0],
        [17335,  3634,   337,  ...,     0,     0,     0]], device='cuda:0')
no proposal
no proposal
['15', '16', '23', '24', '4']
tensor([[   20,  3984,   889,  ...,     0,     0,     0],
        [11087,     5,  2131,  ...,     0,     0,     0],
      

In [ ]:
# trainset = DataIterator(train_instances, args)
for i in range(trainset.batch_count):
    sentence_ids, tokens, masks, token_ranges, tags = trainset.get_batch(i)
    print(tokens)
    refine(tokens)
    # print(refine(tokens).shape)
    print(refine(tokens))

    break

tensor([[  404,    38,    64,  ...,     0,     0,     0],
        [   20, 28287,    21,  ...,     0,     0,     0],
        [   20,   689,    21,  ...,     0,     0,     0],
        ...,
        [   51,   222,   492,  ...,     0,     0,     0],
        [ 1801,  1359,    62,  ...,     0,     0,     0],
        [ 2647,  2156,    42,  ...,     0,     0,     0]], device='cuda:0')


TypeError: Refine.forward() missing 1 required positional argument: 'masks'